In [1]:
import os
import yfinance as yf
from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from fredapi import Fred
from dotenv import load_dotenv

In [2]:
load_dotenv()
FRED_API_KEY = os.environ.get("FRED_API_KEY")

In [3]:
spy = yf.Ticker("SPY")
df_spy = spy.history(start="2000-01-01", end=None)

In [4]:
# c = CurrencyRates()

# start_date = datetime(2025, 1, 1)
# end_date = datetime.today()
# date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# df = pd.DataFrame(index=date_range, columns=["USD to Gold (XAU)"])

# for date in tqdm(date_range, desc="Fetching gold prices"):
#     try:
#         rate = c.get_rate('USD', 'XAU', date)
#         df.at[date, "USD to Gold (XAU)"] = rate
#     except Exception:
#         df.at[date, "USD to Gold (XAU)"] = None

# df = df.astype(float)

# df.head()

In [5]:
fred = Fred(api_key=FRED_API_KEY)

indicators = {
    "CPI": "CPIAUCSL",  # Consumer Price Index (Inflation)
    "Unemployment Rate": "UNRATE",  # US Unemployment Rate
    "GDP Growth": "A191RL1Q225SBEA",  # Real GDP Growth Rate
    "Fed Funds Rate": "FEDFUNDS",  # Federal Funds Interest Rate
    "M2 Money Supply": "M2SL",  # M2 Money Stock
}

start_date = "2000-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")

economic_data = {}
for name, series_id in tqdm(indicators.items(), desc="Fetching Fed Data"):
    data = fred.get_series(series_id, start_date,
                           end_date)
    economic_data[name] = data

df_fred = pd.DataFrame(economic_data)

Fetching Fed Data:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching Fed Data: 100%|██████████| 5/5 [00:01<00:00,  4.26it/s]


In [6]:
df_spy

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
Date,,,,,,,,
2000-01-03 00:00:00-05:00,94.485439,94.485439,91.697083,92.692924,8164300,0.0,0.0,0.0
2000-01-04 00:00:00-05:00,91.478006,91.816592,88.998361,89.068069,8089800,0.0,0.0,0.0
2000-01-05 00:00:00-05:00,89.187568,90.203326,87.474721,89.227402,12177900,0.0,0.0,0.0
2000-01-06 00:00:00-05:00,88.988383,90.183392,87.793373,87.793373,6227200,0.0,0.0,0.0
2000-01-07 00:00:00-05:00,89.426591,92.892120,89.267256,92.892120,8066500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-01-27 00:00:00-05:00,594.809998,599.690002,594.640015,599.369995,70361100,0.0,0.0,0.0
2025-01-28 00:00:00-05:00,600.619995,605.369995,597.250000,604.520020,44433300,0.0,0.0,0.0
2025-01-29 00:00:00-05:00,603.719971,604.130005,599.219971,601.809998,37177400,0.0,0.0,0.0


In [7]:
df_fred

,CPI,Unemployment Rate,GDP Growth,Fed Funds Rate,M2 Money Supply
2000-01-01,169.300,4.0,1.5,5.45,4667.6
2000-02-01,170.000,4.1,NaN,5.73,4680.9
2000-03-01,171.000,4.0,NaN,5.85,4711.7
2000-04-01,170.900,3.8,7.5,6.02,4767.8
2000-05-01,171.200,4.0,NaN,6.27,4755.7
...,...,...,...,...,...
2024-08-01,314.121,4.2,NaN,5.33,21141.3
2024-09-01,314.686,4.1,NaN,5.13,21222.8
2024-10-01,315.454,4.1,2.3,4.83,21312.5
2024-11-01,316.441,4.2,NaN,4.64,21448.3


In [8]:
df_spy.index = pd.to_datetime(df_spy.index).strftime("%Y-%m-%d")

df_spy.index = pd.to_datetime(df_spy.index)
df_fred.index = pd.to_datetime(df_fred.index)

df_merged = pd.merge(df_spy, df_fred, left_index=True,
                     right_index=True, how="outer")

df_merged.index = df_merged.index.strftime("%Y-%m-%d")

df_merged

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains,CPI,Unemployment Rate,GDP Growth,Fed Funds Rate,M2 Money Supply
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169.3,4.0,1.5,5.45,4667.6
2000-01-03,94.485439,94.485439,91.697083,92.692924,8164300.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2000-01-04,91.478006,91.816592,88.998361,89.068069,8089800.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2000-01-05,89.187568,90.203326,87.474721,89.227402,12177900.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2000-01-06,88.988383,90.183392,87.793373,87.793373,6227200.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-27,594.809998,599.690002,594.640015,599.369995,70361100.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2025-01-28,600.619995,605.369995,597.250000,604.520020,44433300.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2025-01-29,603.719971,604.130005,599.219971,601.809998,37177400.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2025-01-30,603.960022,606.599976,600.719971,605.039978,39281300.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [9]:
df_merged = df_merged.drop(
    ["Dividends", "Stock Splits", "Capital Gains", "GDP Growth"], axis=1)

df_merged

,Open,High,Low,Close,Volume,CPI,Unemployment Rate,Fed Funds Rate,M2 Money Supply
2000-01-01,NaN,NaN,NaN,NaN,NaN,169.3,4.0,5.45,4667.6
2000-01-03,94.485439,94.485439,91.697083,92.692924,8164300.0,NaN,NaN,NaN,NaN
2000-01-04,91.478006,91.816592,88.998361,89.068069,8089800.0,NaN,NaN,NaN,NaN
2000-01-05,89.187568,90.203326,87.474721,89.227402,12177900.0,NaN,NaN,NaN,NaN
2000-01-06,88.988383,90.183392,87.793373,87.793373,6227200.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2025-01-27,594.809998,599.690002,594.640015,599.369995,70361100.0,NaN,NaN,NaN,NaN
2025-01-28,600.619995,605.369995,597.250000,604.520020,44433300.0,NaN,NaN,NaN,NaN
2025-01-29,603.719971,604.130005,599.219971,601.809998,37177400.0,NaN,NaN,NaN,NaN
2025-01-30,603.960022,606.599976,600.719971,605.039978,39281300.0,NaN,NaN,NaN,NaN


In [10]:
df_merged[['CPI', 'Unemployment Rate', 'Fed Funds Rate', 'M2 Money Supply']] = df_merged[[
    'CPI', 'Unemployment Rate', 'Fed Funds Rate', 'M2 Money Supply']].fillna(method='ffill')

df_merged

/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_34540/1768084786.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_merged[['CPI', 'Unemployment Rate', 'Fed Funds Rate', 'M2 Money Supply']] = df_merged[[


,Open,High,Low,Close,Volume,CPI,Unemployment Rate,Fed Funds Rate,M2 Money Supply
2000-01-01,NaN,NaN,NaN,NaN,NaN,169.300,4.0,5.45,4667.6
2000-01-03,94.485439,94.485439,91.697083,92.692924,8164300.0,169.300,4.0,5.45,4667.6
2000-01-04,91.478006,91.816592,88.998361,89.068069,8089800.0,169.300,4.0,5.45,4667.6
2000-01-05,89.187568,90.203326,87.474721,89.227402,12177900.0,169.300,4.0,5.45,4667.6
2000-01-06,88.988383,90.183392,87.793373,87.793373,6227200.0,169.300,4.0,5.45,4667.6
...,...,...,...,...,...,...,...,...,...
2025-01-27,594.809998,599.690002,594.640015,599.369995,70361100.0,317.685,4.1,4.48,21533.8
2025-01-28,600.619995,605.369995,597.250000,604.520020,44433300.0,317.685,4.1,4.48,21533.8
2025-01-29,603.719971,604.130005,599.219971,601.809998,37177400.0,317.685,4.1,4.48,21533.8
2025-01-30,603.960022,606.599976,600.719971,605.039978,39281300.0,317.685,4.1,4.48,21533.8


In [11]:
df_cleaned = df_merged.dropna(
    subset=['Open', 'High', 'Low', 'Close', 'Volume'])

df_cleaned

,Open,High,Low,Close,Volume,CPI,Unemployment Rate,Fed Funds Rate,M2 Money Supply
2000-01-03,94.485439,94.485439,91.697083,92.692924,8164300.0,169.300,4.0,5.45,4667.6
2000-01-04,91.478006,91.816592,88.998361,89.068069,8089800.0,169.300,4.0,5.45,4667.6
2000-01-05,89.187568,90.203326,87.474721,89.227402,12177900.0,169.300,4.0,5.45,4667.6
2000-01-06,88.988383,90.183392,87.793373,87.793373,6227200.0,169.300,4.0,5.45,4667.6
2000-01-07,89.426591,92.892120,89.267256,92.892120,8066500.0,169.300,4.0,5.45,4667.6
...,...,...,...,...,...,...,...,...,...
2025-01-27,594.809998,599.690002,594.640015,599.369995,70361100.0,317.685,4.1,4.48,21533.8
2025-01-28,600.619995,605.369995,597.250000,604.520020,44433300.0,317.685,4.1,4.48,21533.8
2025-01-29,603.719971,604.130005,599.219971,601.809998,37177400.0,317.685,4.1,4.48,21533.8
2025-01-30,603.960022,606.599976,600.719971,605.039978,39281300.0,317.685,4.1,4.48,21533.8


In [12]:
df_cleaned['day_diff'] = df_cleaned['Close'] - df_cleaned['Open']

df_cleaned = df_cleaned.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])

df_cleaned

/var/folders/15/vdzhjh015bg5yt68s5n8y4bc0000gn/T/ipykernel_34540/2881686204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['day_diff'] = df_cleaned['Close'] - df_cleaned['Open']


,CPI,Unemployment Rate,Fed Funds Rate,M2 Money Supply,day_diff
2000-01-03,169.300,4.0,5.45,4667.6,-1.792515
2000-01-04,169.300,4.0,5.45,4667.6,-2.409937
2000-01-05,169.300,4.0,5.45,4667.6,0.039834
2000-01-06,169.300,4.0,5.45,4667.6,-1.195010
2000-01-07,169.300,4.0,5.45,4667.6,3.465529
...,...,...,...,...,...
2025-01-27,317.685,4.1,4.48,21533.8,4.559998
2025-01-28,317.685,4.1,4.48,21533.8,3.900024
2025-01-29,317.685,4.1,4.48,21533.8,-1.909973
2025-01-30,317.685,4.1,4.48,21533.8,1.079956


In [13]:
df_cleaned.to_csv("CleanedData.csv")